In [21]:
# $ conda install -c conda-forge ipyvolume
import ewald_summation as es
import matplotlib.pyplot as plt
import numpy as np
import ipyvolume as ipv

In [17]:
def initializer(n_particles, n_dim):
    masses = np.array([39.948] * n_particles)
    q_0 = np.arange(n_particles * n_dim).reshape(n_particles, n_dim)
    v_0 = np.zeros((n_particles, n_dim))
    particle_types = [0] * n_particles
    lj_mixing_conditions = tuple([(3.405, 3.405**6, 0.238)])
    n_particles_tpyes = 1
    return q_0, v_0 * masses[:, None], masses, particle_types, lj_mixing_conditions, n_particles_tpyes

def random_initializer(n_particles, n_dim):
    masses = np.array([39.948] * n_particles)
    q_0 = np.random.random((n_particles, n_dim)) * 20
    v_0 = np.zeros((n_particles, n_dim))
    particle_types = [0] * n_particles
    lj_mixing_conditions = tuple([(3.405, 3.405**6, 0.238)])
    n_particles_tpyes = 1
    return q_0, v_0 * masses[:, None], masses, particle_types, lj_mixing_conditions, n_particles_tpyes

cutoff = 3.405 * 3.5
damping = 0.01
n_steps = 10000
n_particles = 1000
n_dim = 3

In [15]:
q_cuda, p_cuda = es.potentials.lagevin_harmonic_lj_cuda(*initializer(n_particles, n_dim),
                                                        n_steps, cutoff, time_step=0.001, damping=damping)

In [16]:
ipv.figure()
s = ipv.scatter(q_cuda[:, :, 0], q_cuda[:, :, 1], q_cuda[:, :, 2], marker='sphere', size=10)
ipv.xyzlim(-10, 10)
ipv.animation_control(s) # shows controls for animation controls
ipv.show()

In [25]:
n_particles = 1000
n_steps = 50000
q_cuda, p_cuda = es.potentials.lagevin_harmonic_lj_cuda(*random_initializer(n_particles, n_dim),
                                                        n_steps, cutoff, damping=damping, time_step=0.001)


In [26]:
ipv.figure()
s = ipv.scatter(q_cuda[:, :, 0], q_cuda[:, :, 1], q_cuda[:, :, 2], marker='sphere', size=10)
ipv.xyzlim(0, 20)
ipv.animation_control(s) # shows controls for animation controls
ipv.show()